In [1]:
#imports
import numpy as np, pandas as pd, os, openpyxl, re

In [2]:
with open("overheadperf.txt","r") as f:
    txt = list(filter(lambda x:not re.match(r"\s*$",x), f.read().split("\n-------end session-------\n")))

df will be for the first part of the overhead experiment, comparing throughput of all the locks in HomeQueue with throughput of LockFree for all W. The "LockFree" column here is the throughput of LockFree, while the rest are ratios throughput to LockFree.

df1 will be to compare the worker rates of all the parallel versions to the worker rates of Serial-Queue gotten from HW2 for all W.

Runtime was capped at 2000ms and 5 trials of every setting done and averaged out.

In [3]:
df = pd.DataFrame(columns=['W','LockFree','TASLock','pthread_mutex','ALock','CLHLock'])
df.loc[:,'W'] = [25, 50, 100, 200, 400, 800]
df.set_index('W',inplace=True)
df.loc[:,:] = np.zeros((6,5))

In [4]:
df1 = pd.DataFrame(columns=['W','LockFree','TASLock','pthread_mutex','ALock','CLHLock'])
df1.loc[:,'W'] = [25, 50, 100, 200, 400, 800]
df1.set_index('W',inplace=True)
df1.loc[:,:] = np.zeros((6,5))

In [5]:
typedict = {-1:'LockFree', 0:'TASLock', 1:'pthread_mutex', 2:'ALock', 3:'CLHLock'}
wavg = [1144.938891, 863.9249239, 570.8353013, 340.4621775, 189.7636581, 100.0342554] #serial-queue worker rates for W=25,50,100,200,400

In [6]:
#parsing
for s in range(len(txt)):
    W = int(re.search(r'(?<=pkwk: )[0-9]+',txt[s]).group())
    processed = int(re.search(r"(?<=processed: )[0-9]+",txt[s]).group())
    locktype = int(re.search(r'(?<=lockmode: )\-*[0-9]+',txt[s]).group())
    df.loc[W, typedict[locktype]] += processed
    df1.loc[W, typedict[locktype]] += processed

In [7]:
#here and below: setting dfs to throughput avg ratio
df = df/(5*2000)
df1 = df1/(5*2000)

In [8]:
for i in range(6):
    df1.iloc[i,0] /= wavg[i]
    for j in range(1,5):
        df.iloc[i,j] /= df.iloc[i,0]
        df1.iloc[i,j] /= wavg[i]

In [9]:
#write to excel
with pd.ExcelWriter('overEX.xlsx') as writer:
    df.to_excel(writer,sheet_name='Overhead to LockFree')
    df1.to_excel(writer,sheet_name='Worker Rate vs. Serial-Queue')

In [10]:
df

,LockFree,TASLock,pthread_mutex,ALock,CLHLock
W,,,,,
25,2382.9148,0.888583,0.850449,0.878418,0.978977
50,1852.2476,0.941302,0.916282,0.930324,0.949323
100,1520.3800,0.966501,0.941462,0.956947,0.971371
200,1123.3679,0.978601,0.932575,0.966585,0.976139
400,736.7183,0.981725,0.968505,0.979656,0.982759
800,434.1277,0.989918,0.983766,0.989012,0.990380


In [11]:
df1

,LockFree,TASLock,pthread_mutex,ALock,CLHLock
W,,,,,
25,2.081259,1.849371,1.770004,1.828216,2.037506
50,2.143991,2.018144,1.964501,1.994607,2.035339
100,2.663430,2.574207,2.507518,2.548763,2.587180
200,3.299538,3.228931,3.077068,3.189283,3.220807
400,3.882294,3.811346,3.760020,3.803311,3.815359
800,4.339790,4.296038,4.269337,4.292105,4.298044
